In [ ]:
from __future__ import print_function
import os,sys
import ROOT as rt
from larlite import larlite
import chart_studio.plotly as py
import plotly.graph_objects as go
import numpy as np

In [ ]:
f = rt.TFile("icarus_larlite_geodata.root")
t = f.Get("geodata")

In [ ]:
t.GetEntry(0)

In [ ]:
t.cryo_v.at(0)

In [ ]:
def get_tpc_bounds(cryo_v,color=(0,0,0)):
    tpc_plots = []
    for icryo in range(cryo_v.size()):
        cryogeo = cryo_v.at(icryo)
        ntpcs = cryogeo.tpc_v.size()
        for itpc in range(ntpcs):
            #print("tpc[%d]"%(itpc))
            # the four faces
            tpcgeo = cryogeo.tpc_v.at(itpc)
            bounds = np.zeros((3,2))
            for i in range(3):
                bounds[i,0] = tpcgeo.fBounds.at(0)(i)
                bounds[i,1] = tpcgeo.fBounds.at(1)(i)
            #print(bounds)
            top_pts  = [ [bounds[0,0],bounds[1,1], bounds[2,0]],
                          [bounds[0,1],bounds[1,1], bounds[2,0]],
                          [bounds[0,1],bounds[1,1], bounds[2,1]],
                          [bounds[0,0],bounds[1,1], bounds[2,1]],
                          [bounds[0,0],bounds[1,1], bounds[2,0]] ]
            bot_pts  = [ [bounds[0,0],bounds[1,0], bounds[2,0]],
                          [bounds[0,1],bounds[1,0], bounds[2,0]],
                          [bounds[0,1],bounds[1,0], bounds[2,1]],
                          [bounds[0,0],bounds[1,0], bounds[2,1]],
                          [bounds[0,0],bounds[1,0], bounds[2,0]] ]
            up_pts   = [ [bounds[0,0],bounds[1,0], bounds[2,0]],
                          [bounds[0,1],bounds[1,0], bounds[2,0]],
                          [bounds[0,1],bounds[1,1], bounds[2,0]],
                          [bounds[0,0],bounds[1,1], bounds[2,0]],
                          [bounds[0,0],bounds[1,0], bounds[2,0]] ]
            down_pts = [ [bounds[0,0],bounds[1,0], bounds[2,1]],
                          [bounds[0,1],bounds[1,0], bounds[2,1]],
                          [bounds[0,1],bounds[1,1], bounds[2,1]],
                          [bounds[0,0],bounds[1,1], bounds[2,1]],
                          [bounds[0,0],bounds[1,0], bounds[2,1]] ]        
            
            Xe = []
            Ye = []
            Ze = []

            for boundary in [top_pts,bot_pts,up_pts,down_pts]:
                for ipt, pt in enumerate(boundary):
                    Xe.append( pt[0] )
                    Ye.append( pt[1] )
                    Ze.append( pt[2] )
                Xe.append(None)
                Ye.append(None)
                Ze.append(None)
            lines = {
                "type": "scatter3d",
                "x": Xe,
                "y": Ye,
                "z": Ze,
                "mode": "lines",
                "name": "",
                "line": {"color": "rgb(%d,%d,%d)"%color, "width": 5},
            }
            tpc_plots.append(lines)
    return tpc_plots


In [ ]:
detlines = get_tpc_bounds(t.cryo_v)

In [ ]:
axis_template = {
    "showbackground": True,
    "backgroundcolor": "rgba(100, 100, 100,0.5)",
    "gridcolor": "rgb(50, 50, 50)",
    "zerolinecolor": "rgb(0, 0, 0)",
}


layout = go.Layout(
    title='ICARUS TPC',
    autosize=True,
    hovermode='closest',
    showlegend=False,
    scene= {
        "xaxis": axis_template,
        "yaxis": axis_template,
        "zaxis": axis_template,
        "aspectratio": {"x": 2, "y": 1, "z": 3},
        "camera": {"eye": {"x": -3, "y": 0.1, "z": 0.0},
                   "center":dict(x=0, y=0, z=0),
                   "up":dict(x=0, y=1, z=0)},
        "annotations": [],
    }
)

fig = go.Figure(data=detlines, layout=layout)
fig.show()
 